In [1]:
# Get the dataset 
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(
X_train_full, y_train_full)

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
# scale the features -> all are numeric

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# Sequential and Non Sequential Models

In [4]:
# Sequential model with one output

model = keras.models.Sequential([
keras.layers.Dense(30, activation="relu",input_shape=X_train.shape[1:]),
# one output
keras.layers.Dense(1)
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                270       
                                                                 
 dense_1 (Dense)             (None, 1)                 31        
                                                                 
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20,
validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = model.predict(X_new)


Epoch 1/20
363/363 [==============================] - 2s 4ms/step - loss: 0.8280 - val_loss: 0.9038
Epoch 2/20
363/363 [==============================] - 1s 3ms/step - loss: 0.6854 - val_loss: 0.5985
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5658 - val_loss: 1.6154
Epoch 4/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5154 - val_loss: 0.4559
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4437 - val_loss: 0.4295
Epoch 6/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4330 - val_loss: 0.4380
Epoch 7/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4262 - val_loss: 0.4044
Epoch 8/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4150 - val_loss: 0.4089
Epoch 9/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4089 - val_loss: 0.4046
Epoch 10/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4083 - val_loss: 0.4056

# Tuning

In [10]:
# This mimics scikit regressor; so we can do randomsearchcv etc

def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3,input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [11]:
# wrapper for sci-kit, now it can use fit, score and predict

keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

C:\Users\91760\AppData\Local\Temp/ipykernel_7124/1357716075.py:3: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


In [16]:
from scipy.stats import reciprocal
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
}
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10,cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=5,
validation_data=(X_valid, y_valid),
callbacks=[keras.callbacks.EarlyStopping(patience=10)])

C:\Users\91760\anaconda3\envs\tensorflow\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/5
242/242 [==============================] - 1s 4ms/step - loss: 4.8267 - val_loss: 3.1822
Epoch 2/5
242/242 [==============================] - 1s 3ms/step - loss: 1.7499 - val_loss: 1.2972
Epoch 3/5
242/242 [==============================] - 1s 3ms/step - loss: 0.9429 - val_loss: 0.8132
Epoch 4/5
242/242 [==============================] - 1s 3ms/step - loss: 0.7201 - val_loss: 0.6745
Epoch 5/5
121/121 [==============================] - 0s 2ms/step - loss: 0.5797
Epoch 1/5


C:\Users\91760\anaconda3\envs\tensorflow\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 1s 3ms/step - loss: 3.8423 - val_loss: 2.1584
Epoch 2/5
242/242 [==============================] - 1s 3ms/step - loss: 1.4855 - val_loss: 1.1201
Epoch 3/5
242/242 [==============================] - 1s 3ms/step - loss: 0.9239 - val_loss: 0.8392
Epoch 4/5
242/242 [==============================] - 1s 3ms/step - loss: 0.7661 - val_loss: 0.7493
Epoch 5/5
121/121 [==============================] - ETA: 0s - loss: 0.717 - 0s 2ms/step - loss: 0.7116


C:\Users\91760\anaconda3\envs\tensorflow\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


...

C:\Users\91760\anaconda3\envs\tensorflow\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 1s 3ms/step - loss: 1.4327 - val_loss: 0.8062
Epoch 2/5
242/242 [==============================] - 1s 3ms/step - loss: 0.7386 - val_loss: 0.7146
Epoch 3/5
242/242 [==============================] - 1s 3ms/step - loss: 0.6639 - val_loss: 0.6483
Epoch 4/5
242/242 [==============================] - 1s 3ms/step - loss: 0.6188 - val_loss: 0.6091
Epoch 5/5
121/121 [==============================] - 0s 2ms/step - loss: 0.6165


C:\Users\91760\anaconda3\envs\tensorflow\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [-0.64860596 -0.40896992 -0.54830353 -0.42037062 -0.56252042 -0.44318431
         nan -0.70167126 -0.55877342 -0.58155352]
  warnings.warn(


Epoch 1/5
363/363 [==============================] - 2s 3ms/step - loss: 0.7534 - val_loss: 1.5683
Epoch 2/5
363/363 [==============================] - 1s 3ms/step - loss: 0.4661 - val_loss: 0.4465
Epoch 3/5
363/363 [==============================] - 1s 3ms/step - loss: 0.4074 - val_loss: 0.3744
Epoch 4/5
363/363 [==============================] - 1s 3ms/step - loss: 0.3795 - val_loss: 0.3668
Epoch 5/5
363/363 [==============================] - 1s 3ms/step - loss: 0.3683 - val_loss: 0.3692


RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x000001AC5AAAD2B0>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001AC529211F0>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [17]:
rnd_search_cv.best_params_


{'learning_rate': 0.016880878822421123, 'n_hidden': 2, 'n_neurons': 24}